In [23]:
import torch
import torch.nn as nn

In [24]:
# Creamos la clase que conforma el bloque convolucional del modelo

class BlockConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BlockConv, self).__init__()
        self.blockConv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding="same"),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(negative_slope=0.01,inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding="same"),  # Segunda capa de convolución
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
        )

    def forward(self, x):
        return self.blockConv(x)

In [25]:
# Definimos la clase con la que se va a abordar el downsamplig

class DownsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownsamplingBlock, self).__init__()
        self.blockConv = BlockConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, return_indices=True)

    def forward(self, x):
        skip_out = self.blockConv(x)
        down_out, indices = self.down_sample(skip_out)
        #print(f"Downsampling: down_out shape: {down_out.shape}, indices shape: {indices.shape}")
        return (down_out, skip_out, indices)

In [26]:
# Definimos la clase con la que se aborda el upsampling

class UpsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, upsample_tec):
        super(UpsamplingBlock, self).__init__()
        self.upsample_tec = upsample_tec
        if self.upsample_tec == 'conv_transpose':
            self.upsamplingBlock = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)
        elif self.upsample_tec == 'bilinear':
            self.upsamplingBlock = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        elif self.upsample_tec == 'maxunpooling':
            self.upsamplingBlock = nn.MaxUnpool2d(kernel_size=2, stride=2)
        else:
            raise ValueError("Los métodos de Upsampling aceptados son la convolución transpuesta, la interpolación bilineal y el max unpooling")

        self.double_conv = BlockConv(in_channels, out_channels)

    def forward(self, down_input, skip_input, indices):
        if self.upsample_tec == 'maxunpooling':
            #print(f"Upsampling: down_input shape: {down_input.shape}, indices shape: {indices.shape}")
            x = self.upsamplingBlock(down_input, indices)
        else:
            x = self.upsamplingBlock(down_input)
        x = torch.cat([x, skip_input], dim=1)

        return self.double_conv(x)

## Clase UNET

##### Implementa la arquitectura del modelo UNET que vamos a utilizar para la realización de este Trabajo Fin de Grado

##### La clase tiene dos parámetros:
##### - "out_labels": se especifíca el número de clases de salida para la tarea de segmentación. En nuestro caso el conjunto de datos esta preparado para que el modelo diferencie 5 clases diferentes. Por lo que el modelo generará 5 mapas de características diferentes, uno para cada clase.
##### - "upsample_tec": nos permite elegir la técnica de upsampling que queremos aplicar en nuestro modelo. Si introducimos el valor "conv_transpose" realizará operaciones de convolución transpuesta, con el valor "bilinear" llevará a cabo una interpolación bilinear y si le asignamos "maxunpooling", llevará a cabo max unpooling. 

##### Como vemos la arquitectura de nuestro modelo consta de varias partes:

##### - Downsampling: capas de convolución en las que se obtienen mapas de características que extraen patrones de la imagen original, estos son de menor dimensiones que la imagen original. Hay 4 capas de estas: down_1, down_2, down_3 y down_4.

##### - Cuello de botella: esta es la capa denominada "neck_conv", y sirve de unión entre la fase de extracción y la de expansión.

##### - Upsampling: capas de reconstrucción de la imagen original, y en cada capa de "UpsamplingBlock" se incrementa la resolución de la imagen. Hay 4 capas de estas: up_1, up_2, up_3 y up_4.

##### - Convolución final: denominada "last_conv", es la última capa, esta reduce el número de canales a tantos como clases finales deseemos. Se consigue con un kernel de tamaño 1x1

In [27]:
class UNet(nn.Module):
    def __init__(self, out_classes, upsample_tec):
        super(UNet, self).__init__()
        self.upsample_tec = upsample_tec
        # Downsampling
        # 3 canales de informacion --> imagenes RGB
        self.down_1 = DownsamplingBlock(3, 64)
        self.down_2 = DownsamplingBlock(64, 128)
        self.down_3 = DownsamplingBlock(128, 256)
        self.down_4 = DownsamplingBlock(256, 512)
        
        # Cuello de botella
        self.neck_conv = BlockConv(512, 1024)
        
        # Upsampling
        self.up_4 = UpsamplingBlock(512 + 1024, 512, self.upsample_tec)
        self.up_3 = UpsamplingBlock(256 + 512, 256, self.upsample_tec)
        self.up_2 = UpsamplingBlock(128 + 256, 128, self.upsample_tec)
        self.up_1 = UpsamplingBlock(64 + 128, 64, self.upsample_tec)
        
        # Dropout
        self.droput = nn.Dropout(0.35)
        
        # Final Convolution
        # Genera tantos planos como clases tenemos que identificar
        self.last_conv = nn.Conv2d(64, out_classes, kernel_size=1)

    # La función forward describe el flujo de los datos a través de la red
    # Capas de downsampling --> CUello de botella --> Capas de upsampling --> Salida final
    def forward(self, x):
        x, skip1_out,idx1  = self.down_1(x)
        x, skip2_out,idx2  = self.down_2(x)
        x, skip3_out,idx3  = self.down_3(x)
        x, skip4_out,idx4  = self.down_4(x)
        
        x = self.neck_conv(x)
        
        x = self.up_4(x, skip4_out, idx4)
        x = self.up_3(x, skip3_out, idx3)
        x = self.up_2(x, skip2_out, idx2)
        x = self.up_1(x, skip1_out, idx1)
        
        x = self.last_conv(x)
        
        return x